## Model Version 1

In [1]:
! pip install pyomo

## Create Model

In [370]:
# Import of the pyomo module
from pyomo.environ import *
 
# Creation of a Concrete Model
model = ConcreteModel()

## Declare Constant

In [371]:
# declare constant
Max_Capital_Cost=15000000
Mile_Cost=4
Truck_Size=6000
###need input
Load_Cost=0.46
###need input
Unload_Cost=0.46
Hops=2
Fixed_Trucks_Cost=550

M=1000000000000



## Declare Set

In [372]:
# declare set
## Define sets ##
#  Sets
#       L   All location   / Syracuse, NY, Buffalo, Elizabeth, Ithaca /
#       I   Customer warehouse          / Syracuse / ;
#       J   PB facility   /  NY, Buffalo, Elizabeth /
#       K   USPS or Courier hubs          / Ithaca / ;
#       I   Customer warehouse          / Syracuse / ;
#       h   hops day  /  0,1,2 / ;
#       S   PB facility sizes         / small, medium, large / ;
#       T   PB facility types   /  deliver, return, both / ;
#       F   PB facility features         / ingest, both / ;


model.L=Set(initialize=['Syracuse','NY','Buffalo','Elizabeth','Ithaca'])
model.I=Set(initialize=['Syracuse'])
model.J=Set(initialize=['NY','Buffalo','Elizabeth'])
model.K=Set(initialize=['Ithaca'])
model.S=Set(initialize=['small','medium','large'])
model.T=Set(initialize=['deliver','return','both'])
model.F=Set(initialize=['ingest','both'])



## Declare Parameters

In [373]:
## Define parameters ##
#   Parameters
#       capacity(j,s,t,f)  amount of capital needed to build a hub at location J, size S, type T, feature F;
#       utility(j,s,t,f) Operating cost for a hub;
#       distance(l,l') distance between location l and location l';
#       hub_capacity(s) hub capacity of different size;
#       deliver(i,k)  the number of deliver packages from i to k;
#       repackage(k,i)  the number of return packages from k to i;
#       sorting(j)      the sorting cost in location j

## for loop create capacity_tab when data is available
#capacity_tab={('NY','small','deliver', 'ingest'):10000}
def capacity_init(model, J, S, T, F):
    if S=="small":
        return 1000000
    elif S=="medium":
        return 4000000
    elif S=="large":
        return 10000000
    #return 20000
model.capital = Param(model.J,model.S,model.T, model.F,initialize=capacity_init)

## for loop create utility_tab when data is available
#utility_tab={('NY','small','deliver', 'ingest'):10000}
def utility_init(model, J, S, T, F):
    return 500
model.utility = Param(model.J,model.S,model.T, model.F,initialize=utility_init)

## for loop create dis_tab when data is available
#dis_tab={('NY','Buffalo'):600,('NY','Elizabeth'):100,('Ithaca','NY'):500}
def dis_init(model, L, L1):
    return 10
model.distance = Param(model.L,model.L,initialize=dis_init)

## Hub Capacity
model.hub_capacity=Param(model.S,initialize={'small':4800000,'medium':14400000,'large':28800000})

def package(model, l, l1):
    return 0
model.package = Param(model.I|model.K,model.I|model.K,initialize=package)

def max_trans_cost_init(model,l,l1):
    return max(Fixed_Trucks_Cost, model.distance[l,l1]*Mile_Cost)
model.max_trans_cost = Param(model.L,model.L,initialize = max_trans_cost_init)
## The number of deliver packages from i to j
## for loop create deliver_tab
#deliver_tab={('Syracuse','Ithaca'):0}
#def deliver_init(model, I,K):
#    return 0
#model.deliver = Param(model.I,model.K,initialize=deliver_init)

## The number of deliver packages from i to j
## for loop create deliver_tab
#repackage_tab={('Ithaca','Syracuse'):0}
#def repackage_init(model, K, I):
#    return 0
#model.repackage = Param(model.K,model.I,initialize=repackage_init)

## Sorting cost
def sorting_init(model, J):
    return 10
model.sorting = Param(model.J,initialize=sorting_init)




## Declare Variable

In [374]:
## Define variables ##
#  Variables
#       y(j,s,t,f)  Binary variable indicates whether to open a facility
#       truck(l,l)       the number of truck from l to l ;
#       n(l,l',l'',h)   the number of packages at current location l 
                     #going to location l' next, with a final destination of l'' and h hops left.
model.y=Var(model.J,model.S,model.T,model.F, within=Binary)
model.truck=Var(model.L,model.L, within=NonNegativeIntegers)
model.n=Var(model.L,model.L,model.L,range(Hops+1), within=NonNegativeReals)
#model.max_trans_cost = Var(model.L,model.L, within=NonNegativeReals)

## Objective Function

In [375]:
# operations cost + transportation cost
# operations cost + transportation cost
#(2)
def obj_expression(model):
    utility_cost=sum((model.utility[j,s,t,f] * model.y[j,s,t,f]) for j in model.J for s in model.S for t in model.T for f in model.F)
    loading_cost=sum((Load_Cost * model.n[j,l,l1,h]) for j in model.J for l in model.L for l1 in (model.I|model.K) for h in range(0,Hops+1))
    unloading_cost=sum((Unload_Cost * model.n[l,j,l1,h]) for l in model.L for j in model.J for l1 in (model.I|model.K) for h in range(0,Hops+1)) 
    sorting_cost=sum((model.sorting[j] * model.n[j,k,k,h]) for j in model.J for k in model.K for h in range(0,Hops+1)) 
    transportation_cost=sum((model.max_trans_cost[l,l1] * model.truck[l,l1]) for l in model.L for l1 in model.L) #(3)
    return utility_cost + loading_cost + unloading_cost + sorting_cost + transportation_cost
model.OBJ = Objective(rule=obj_expression, sense = minimize, doc='Define objective function')

In [376]:
model.pprint()

26 Set Declarations
    F : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['both', 'ingest']
    I : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=None
        ['Syracuse']
    J : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['Buffalo', 'Elizabeth', 'NY']
    K : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=None
        ['Ithaca']
    L : Dim=0, Dimen=1, Size=5, Domain=None, Ordered=False, Bounds=None
        ['Buffalo', 'Elizabeth', 'Ithaca', 'NY', 'Syracuse']
    S : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['large', 'medium', 'small']
    T : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['both', 'deliver', 'return']
    capital_index : Dim=0, Dimen=4, Size=54, Domain=None, Ordered=False, Bounds=None
        Virtual
    capital_index_index_0 : Dim=0, Dimen=3, Size=54, Domain=None, Ordered=False, Bounds=None
        Virtual
    capital_index_index_

## Constraints

In [377]:
## Define contrains ##
# capital_constrains   the sum of the capital cost

#(1)
def capital_constraints(model):
    total_cost=0
    for j in model.J:
        for s in model.S:
            for t in model.T:
                for f in model.F:
                    total_cost+=model.capital[j,s,t,f]*model.y[j,s,t,f]
    return total_cost<=Max_Capital_Cost
model.max_cap = Constraint( rule = capital_constraints, doc = "max capital constraint")



In [378]:
#(4)
#def max_trans_cost_rule(model,l,l1):
#    return model.max_trans_cost[l,l1]>=model.distance[l,l1]*Mile_Cost
#model.max_trans_con=Constraint(model.L,model.L,rule=max_trans_cost_rule,doc="transportation cost constraint")

In [379]:
#(5)
#def max_trans_cost_rule2(model,l,l1):
#    return model.max_trans_cost[l,l1]>=Fixed_Trucks_Cost
#model.max_trans_con2=Constraint(model.L,model.L,rule=max_trans_cost_rule2,doc="transportation cost constraint")

In [380]:
#(6)
def truck_size_constraints(model,l,l1):
    trucksize=0
    for l2 in model.I | model.K:
        for h in range(Hops):
            trucksize+=model.n[l,l1,l2,h]
    return trucksize<= Truck_Size*model.truck[l,l1]
model.truck_size_con = Constraint(model.L,model.L,rule = truck_size_constraints, doc = "truck size constraint")

In [381]:
#(7)
def inbound_equal_rule(model, j, h, l1):
      return (sum(model.n[l,j,l1,h] for l in model.L)
          == sum(model.n[j,l2,l1,h-1] for l2 in model.L))
model.inbound_equal = Constraint(model.J,range(1,Hops),model.I|model.K, rule = inbound_equal_rule, doc = "inbound package equal")

In [382]:
#(8)
#2nd demand contransts
def initialization_demand_rule(model,l,l1):
    if l == l1:
        return Constraint.Skip
    return (sum(model.n[l,j,l1,Hops] for j in model.J) == model.package[l,l1])
model.initialization_demand = Constraint(model.I|model.K,model.I|model.K,rule = initialization_demand_rule)

In [383]:
#(9)
def destination_demand_rule(model, l):
    return (sum(model.n[j,l,l,h] for j in model.J for h in range(0,Hops+1)) == sum(model.package[l1,l] for l1 in model.I|model.K))
model.destination_demand_rule = Constraint(model.I|model.K, rule=destination_demand_rule)

In [384]:
#(10)
def num_of_hubs_opened_perlocation(model,j):
    num=0
    for s in model.S:
        for t in model.T:
            for f in model.F:
                num=model.y[j,s,t,f]
                return num<=1
model.num=Constraint(model.J,rule=num_of_hubs_opened_perlocation,doc="contraint on num of hubs opened per location")

In [385]:
#(11)
def delivery_hub_rule(model,j,k,l):
    return (sum(model.n[l,j,k,h] for h in range(0,Hops+1))
           <= (M * sum(model.y[j,s,'deliver',f]+model.y[j,s,'deliver',f] for s in model.S for f in model.F)))
model.delivery_hub = Constraint(model.J,model.K,model.I|model.J, rule = delivery_hub_rule, doc = "ensure delivery packages handled by delivery hub")

In [386]:
#(12)
def delivery_sort_hub_rule(model,j,k):
    return (sum(model.n[j,k,k,h] for h in range(0,Hops+1))
           <= (M * sum(model.y[j,s,'deliver','both']+model.y[j,s,'return','both'] for s in model.S)))
model.delivery_sort_hub = Constraint(model.J,model.K, rule = delivery_sort_hub_rule, doc = "ensure last delivery packages handled by sort-delivery hub")

In [387]:
#(13)
def return_hub_rule(model,i,j,k,l):
    return (sum(model.n[l,j,i,h] for h in range(0,Hops+1))
           <= (M * sum(model.y[j,s,'return',f]+model.y[j,s,'return',f] for s in model.S for f in model.F)))
model.return_hub = Constraint(model.I,model.J,model.K,model.L, rule = return_hub_rule, doc = "ensure return packages handled by return hub")

In [388]:
#(14)
def hub_capacity_constraint(model,j):
    return sum(model.n[l,j,l1,h] for l in model.L for l1 in model.I|model.K for h in range(0,Hops+1))<=sum(model.hub_capacity[s]*model.y[j,s,t,f] for s in model.S for t in model.T for f in model.F)
model.hub_capacity_constraint=Constraint(model.J, rule=hub_capacity_constraint,doc="hub capacity constraint")

In [390]:
from pyomo.opt import SolverFactory
#import pyomo.environ
opt = SolverFactory("glpk")
results = opt.solve(model)
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
#pyomo_postprocess(None, model, results)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 0.0
  Upper bound: 0.0
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 222
  Number of nonzeros: 681
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.011852741241455078
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------